In [1]:
# Import Pandas library
import pandas as pd
# Upload CSV file from your device
from google.colab import files
uploaded = files.upload()

# Assuming the file is named 'legal_data.csv'
file_name = list(uploaded.keys())[0]

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_name)

Saving filtered_dataset (1).csv to filtered_dataset (1).csv


In [2]:
pip install transformers


In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [4]:
import pandas as pd



summaries = []

for text in df['case_text'].astype(str):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)

df['bart_summaries'] = summaries



In [5]:
df.head()

,case_id,case_title,case_text,case_outcome,bart_summaries
0,Case2,Black v Lipovac [1998] FCA 699 ; (1998) 217 AL...,The general principles governing the exercise ...,cited,The general principles governing the exercise ...
1,Case5,Dr Martens Australia Pty Ltd v Figgins Holding...,The preceding general principles inform the ex...,cited,discretion is not to be fettered by transform...
2,Case9,Australian Broadcasting Corporation v O'Neill ...,recent decision of the High Court in Australia...,applied,High Court in Australian Broadcasting Corporat...
3,Case14,George v Cluning (1979) 53 ALJR 767 (note),"Strictly speaking, a cheque, even a bank chequ...",followed,"Strictly speaking, a cheque is not a form of l..."
4,Case15,Australian Mid-Eastern Club Limited v Yassim (...,None of this is to suggest that the Deputy Com...,followed,The Deputy Commissioner was not under any obli...


In [6]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd

# Assuming 'df' is your DataFrame containing the generated extractive summaries and reference summaries
# Replace 'generated_summary' and 'reference_summary' with the actual column names if they are different
generated_summaries = df['bart_summaries'].astype(str).tolist()
reference_summaries = df['case_text'].astype(str).tolist()

# Initialize lists to store BLEU scores
bleu1_scores = []
bleu2_scores = []
bleu3_scores = []

# Compute BLEU scores for each pair of generated summary and reference summary
for generated_summary, reference_summary in zip(generated_summaries, reference_summaries):
    # Tokenize the summaries into lists of words
    generated_tokens = generated_summary.split()
    reference_tokens = reference_summary.split()

    # Compute BLEU-1, BLEU-2, and BLEU-3 scores
    bleu1 = sentence_bleu([reference_tokens], generated_tokens, weights=(1, 0, 0), smoothing_function=SmoothingFunction().method1)
    bleu2 = sentence_bleu([reference_tokens], generated_tokens, weights=(0.5, 0.5, 0), smoothing_function=SmoothingFunction().method1)
    bleu3 = sentence_bleu([reference_tokens], generated_tokens, weights=(0.33, 0.33, 0.33), smoothing_function=SmoothingFunction().method1)

    # Append the scores to the respective lists
    bleu1_scores.append(bleu1)
    bleu2_scores.append(bleu2)
    bleu3_scores.append(bleu3)

# Add BLEU scores as new columns in the DataFrame
df['bleu1_score'] = bleu1_scores
df['bleu2_score'] = bleu2_scores
df['bleu3_score'] = bleu3_scores

# Optionally, you can also print the average BLEU scores
print("Average BLEU-1 Score:", pd.Series(bleu1_scores).mean())
print("Average BLEU-2 Score:", pd.Series(bleu2_scores).mean())
print("Average BLEU-3 Score:", pd.Series(bleu3_scores).mean())


Average BLEU-1 Score: 0.27256577997357667
Average BLEU-2 Score: 0.2678928842109082
Average BLEU-3 Score: 0.26351241652780844


In [7]:
!pip install rouge


In [9]:
import pandas as pd
from rouge import Rouge

# Assuming 'df' is your DataFrame containing the generated extractive summaries and reference summaries
# Replace 'generated_summary' and 'reference_summary' with the actual column names if they are different
generated_summaries = df['bart_summaries'].astype(str).tolist()
reference_summaries = df['case_text'].astype(str).tolist()

# Initialize the Rouge object
rouge = Rouge()

# Initialize lists to store ROUGE scores
rouge1_scores = []
rouge2_scores = []
rouge3_scores = []

# Compute ROUGE scores for each pair of generated summary and reference summary
for generated_summary, reference_summary in zip(generated_summaries, reference_summaries):
    # Compute ROUGE scores
    scores = rouge.get_scores(generated_summary, reference_summary)[0]

    # Append ROUGE-1 and ROUGE-2 scores
    rouge1_scores.append(scores['rouge-1']['f'])
    rouge2_scores.append(scores['rouge-2']['f'])

    # Check if the 'rouge-3' key exists in the scores dictionary
    if 'rouge-3' in scores:
        rouge3_scores.append(scores['rouge-3']['f'])
    else:
        rouge3_scores.append(scores['rouge-2']['f'])  # Append None if ROUGE-3 cannot be computed

# Add ROUGE scores as new columns in the DataFrame
df['rouge1_score'] = rouge1_scores
df['rouge2_score'] = rouge2_scores
df['rouge3_score'] = rouge3_scores

# Optionally, you can also print the average ROUGE scores
print("Average ROUGE-1 Score:", pd.Series(rouge1_scores).mean())
print("Average ROUGE-2 Score:", pd.Series(rouge2_scores).mean())
print("Average ROUGE-3 Score:", pd.Series(rouge3_scores).mean())


Average ROUGE-1 Score: 0.6427687526306136
Average ROUGE-2 Score: 0.5803562062918887
Average ROUGE-3 Score: 0.5803562062918887


In [10]:
# Calculate the length of each 'case_text' and 'bart_summaries'
df['case_text_length'] = df['case_text'].str.len()
df['bart_summaries_length'] = df['bart_summaries'].str.len()

# Compute the difference in length between 'case_text' and 'bart_summaries'
df['length_difference'] = df['case_text_length'] - df['bart_summaries_length']

# Calculate the average length difference
average_length_difference = df['length_difference'].mean()

print("Average length difference between 'case_text' and 'bart_summaries':", average_length_difference)

Average length difference between 'case_text' and 'bart_summaries': 423.444
